In [1]:
import os
import numpy as np
import cv2
import glob
from PIL import Image
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix

# Configuration
test_folder = r'C:\Users\abudh\Desktop\CropWatch\Test2'
filtered_image_folder = r'C:\Users\abudh\Desktop\CropWatch\Filtered_Image'
model_path = r'C:\Users\abudh\Desktop\CropWatch\Training_model.h5'
img_height = 64
img_width = 64
n = 2  # Number of rows and columns to splice the image into
threshold_percentage = 30  # Threshold for cloud coverage

vegetation_classes = ["AnnualCrop", "Forest", "HerbaceousVegetation", "Pasture", "PermanentCrop"]

# Load trained model
def load_trained_model(model_path):
    model = load_model(model_path)
    print("Model loaded successfully.")
    return model

# Preprocess image for prediction
def preprocess_image(image):
    img = image.resize((img_width, img_height))
    img_array = np.array(img) / 255.0
    return img_array

# Splice image into parts
def splice_image(file_path, n):
    img = Image.open(file_path)
    width, height = img.size
    cropped_images = []
    tile_width = width // n
    tile_height = height // n

    for i in range(n):
        for j in range(n):
            left = j * tile_width
            upper = i * tile_height
            right = left + tile_width
            lower = upper + tile_height
            cropped_img = img.crop((left, upper, right, lower))
            cropped_images.append((cropped_img, i * n + j + 1))
    return cropped_images

# Mask clouds and interpolate
def mask_clouds(image, t1=0.1, t2=0.1, kernel_size=10, inpaint_radius=5):
    # Convert PIL image to NumPy array
    image = np.array(image)
    if image is None:
        raise ValueError("Image conversion failed, image is None")

    R = image[:, :, 2]
    G = image[:, :, 1]
    B = image[:, :, 0]

    rg = R / (G + 1e-6)
    gb = G / (B + 1e-6)

    cloud_mask1 = np.logical_and(np.abs(rg - 1) < t1, np.abs(gb - 1) < t2).astype(np.uint8)
    cloud_mask = cv2.dilate(cloud_mask1, kernel=np.ones((kernel_size, kernel_size), dtype=np.uint8), iterations=1)

    R_inpaint = cv2.inpaint(R, cloud_mask, inpaint_radius, cv2.INPAINT_TELEA)
    G_inpaint = cv2.inpaint(G, cloud_mask, inpaint_radius, cv2.INPAINT_TELEA)
    B_inpaint = cv2.inpaint(B, cloud_mask, inpaint_radius, cv2.INPAINT_TELEA)

    inpainted_image = np.stack([B_inpaint, G_inpaint, R_inpaint], axis=2)

    cloud_cover = np.sum(cloud_mask) / cloud_mask.size * 100
    return cloud_cover


def analyze_image_parts(image_path, n, threshold_percentage):
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    output_folder = filtered_image_folder
    os.makedirs(output_folder, exist_ok=True)
    
    cropped_images = splice_image(image_path, n)
    
    for cropped_img, idx in cropped_images:
        # Comment out the temporary file creation
        # temp_path = os.path.join(output_folder, f"temp_{idx}.png")
        # cropped_img.save(temp_path)
        
        # cloud_cover = mask_clouds(temp_path, t1=0.1, t2=0.1)
        cloud_cover = mask_clouds(cropped_img)  # Directly use cropped_img instead of temp_path
        
        status = "Accepted" if cloud_cover < threshold_percentage else "Rejected"
        print(f"Part {idx} Cloud Coverage: {cloud_cover:.2f}% - {status}")
        
        if cloud_cover < threshold_percentage:
            # Save the image in the appropriate vegetation class folder
            img_array = preprocess_image(cropped_img)
            img_array_exp = np.expand_dims(img_array, axis=0)
            prediction = model.predict(img_array_exp)
            predicted_class = np.argmax(prediction, axis=1)[0]
            predicted_label = vegetation_classes[predicted_class]

            if predicted_label in vegetation_classes:
                class_folder = os.path.join(filtered_image_folder, predicted_label)
                os.makedirs(class_folder, exist_ok=True)
                output_name = f"{base_name}_Q{idx}.png"
                output_path = os.path.join(class_folder, output_name)
                cropped_img.save(output_path)
                print(f"Spliced image {output_name} saved to {output_path}")
        
        # Commented out the removal of the temporary file
        # os.remove(temp_path)

# Classify and save images
def classify_and_save_images(images, model, vegetation_classes, filtered_image_folder, n):
    if not os.path.exists(filtered_image_folder):
        os.makedirs(filtered_image_folder)

    for veg_class in vegetation_classes:
        class_folder = os.path.join(filtered_image_folder, veg_class)
        if not os.path.exists(class_folder):
            os.makedirs(class_folder)

    for file_path, img_array in images:
        try:
            img_array_exp = np.expand_dims(img_array, axis=0)
            prediction = model.predict(img_array_exp)
            predicted_class = np.argmax(prediction, axis=1)[0]
            predicted_label = vegetation_classes[predicted_class]

            if predicted_label in vegetation_classes:
                # Splice the image if it's a vegetation class
                cropped_images = splice_image(file_path, n)
                
                for cropped_img, idx in cropped_images:
                    base_name = os.path.splitext(os.path.basename(file_path))[0]
                    new_filename = f"{base_name}_part{idx}.jpg"
                    save_path = os.path.join(filtered_image_folder, predicted_label, new_filename)

                    cropped_img.save(save_path)
                    print(f"Spliced image {file_path} classified as {predicted_label} and saved to {save_path}")
            else:
                print(f"Image {file_path} classified as {predicted_label} but not a vegetation class.")

        except Exception as e:
            print(f"Error processing image {file_path}: {e}")

# Load and prepare images for prediction
def load_and_prepare_images(folder):
    image_files = glob.glob(os.path.join(folder, '*.*'))
    images = []
    for file in image_files:
        try:
            with Image.open(file) as img:
                img_array = preprocess_image(img)
                images.append((file, img_array))
        except Exception as e:
            print(f"Error loading image {file}: {e}")
    return images

# Main function
model = load_trained_model(model_path)



# Load images for classification
images = load_and_prepare_images(test_folder)

# Classify and save images
classify_and_save_images(images, model, vegetation_classes, filtered_image_folder, n)

# Analyze image parts
image_files = glob.glob(os.path.join(test_folder, '*.*'))
for image_file in image_files:
    analyze_image_parts(image_file, n, threshold_percentage)

print("Complete")


Model loaded successfully.
1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step
Spliced image C:\Users\abudh\Desktop\CropWatch\Test\AnnualCrop_1.jpg classified as HerbaceousVegetation and saved to C:\Users\abudh\Desktop\CropWatch\Filtered_Image\HerbaceousVegetation\AnnualCrop_1_part1.jpg
Spliced image C:\Users\abudh\Desktop\CropWatch\Test\AnnualCrop_1.jpg classified as HerbaceousVegetation and saved to C:\Users\abudh\Desktop\CropWatch\Filtered_Image\HerbaceousVegetation\AnnualCrop_1_part2.jpg
Spliced image C:\Users\abudh\Desktop\CropWatch\Test\AnnualCrop_1.jpg classified as HerbaceousVegetation and saved to C:\Users\abudh\Desktop\CropWatch\Filtered_Image\HerbaceousVegetation\AnnualCrop_1_part3.jpg
Spliced image C:\Users\abudh\Desktop\CropWatch\Test\AnnualCrop_1.jpg classified as HerbaceousVegetation and saved to C:\Users\abudh\Desktop\CropWatch\Filtered_Image\HerbaceousVegetation\AnnualCrop_1_part4.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step
Spliced image C:\Users\abudh\Desktop\CropWatch\Test\AnnualCr

IndexError: list index out of range

In [ ]:
import os
import numpy as np
import cv2
import glob
from PIL import Image
from tensorflow.keras.models import load_model

# Configuration
test_folder = r'C:\Users\abudh\Desktop\CropWatch\Test2'
filtered_image_folder = r'C:\Users\abudh\Desktop\CropWatch\Filtered_Image'
model_path = r'C:\Users\abudh\Desktop\CropWatch\Training_model.h5'
img_height = 64
img_width = 64
n = 3  # Number of rows and columns to splice the image into
threshold_percentage = 30  # Threshold for cloud coverage

vegetation_classes = ["AnnualCrop", "Forest", "HerbaceousVegetation", "Pasture", "PermanentCrop"]

# Load trained model
def load_trained_model(model_path):
    model = load_model(model_path)
    print("Model loaded successfully.")
    
    # Print model summary to check output shape
    model.summary()
    
    return model

# Preprocess image for prediction
def preprocess_image(image):
    img = image.resize((img_width, img_height))
    img_array = np.array(img) / 255.0
    return img_array

# Splice image into parts
def splice_image(file_path, n):
    img = Image.open(file_path)
    width, height = img.size
    cropped_images = []
    tile_width = width // n
    tile_height = height // n

    for i in range(n):
        for j in range(n):
            left = j * tile_width
            upper = i * tile_height
            right = left + tile_width
            lower = upper + tile_height
            cropped_img = img.crop((left, upper, right, lower))
            cropped_images.append((cropped_img, i * n + j + 1))
    return cropped_images

# Mask clouds and interpolate
def mask_clouds(image, t1=0.1, t2=0.1, kernel_size=10, inpaint_radius=5):
    # Convert PIL image to NumPy array
    image = np.array(image)
    if image is None:
        raise ValueError("Image conversion failed, image is None")

    R = image[:, :, 2]
    G = image[:, :, 1]
    B = image[:, :, 0]

    rg = R / (G + 1e-6)
    gb = G / (B + 1e-6)

    cloud_mask1 = np.logical_and(np.abs(rg - 1) < t1, np.abs(gb - 1) < t2).astype(np.uint8)
    cloud_mask = cv2.dilate(cloud_mask1, kernel=np.ones((kernel_size, kernel_size), dtype=np.uint8), iterations=1)

    R_inpaint = cv2.inpaint(R, cloud_mask, inpaint_radius, cv2.INPAINT_TELEA)
    G_inpaint = cv2.inpaint(G, cloud_mask, inpaint_radius, cv2.INPAINT_TELEA)
    B_inpaint = cv2.inpaint(B, cloud_mask, inpaint_radius, cv2.INPAINT_TELEA)

    inpainted_image = np.stack([B_inpaint, G_inpaint, R_inpaint], axis=2)

    cloud_cover = np.sum(cloud_mask) / cloud_mask.size * 100
    return cloud_cover

def analyze_image_parts(image_path, n, threshold_percentage):
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    output_folder = filtered_image_folder
    os.makedirs(output_folder, exist_ok=True)
    
    cropped_images = splice_image(image_path, n)
    
    for cropped_img, idx in cropped_images:
        cloud_cover = mask_clouds(cropped_img)  # Directly use cropped_img
        
        status = "Accepted" if cloud_cover < threshold_percentage else "Rejected"
        print(f"Part {idx} Cloud Coverage: {cloud_cover:.2f}% - {status}")
        
        if cloud_cover < threshold_percentage:
            # Save the image in the appropriate vegetation class folder
            img_array = preprocess_image(cropped_img)
            img_array_exp = np.expand_dims(img_array, axis=0)
            prediction = model.predict(img_array_exp)
            
            # Debug prints
            print(f"Prediction shape: {prediction.shape}")
            print(f"Prediction values: {prediction}")
            
            predicted_class = np.argmax(prediction, axis=1)[0]
            print(f"Predicted class index: {predicted_class}")
            
            if predicted_class < len(vegetation_classes):
                predicted_label = vegetation_classes[predicted_class]
                
                if predicted_label in vegetation_classes:
                    class_folder = os.path.join(filtered_image_folder, predicted_label)
                    os.makedirs(class_folder, exist_ok=True)
                    output_name = f"{base_name}_Q{idx}.png"
                    output_path = os.path.join(class_folder, output_name)
                    cropped_img.save(output_path)
                    print(f"Spliced image {output_name} saved to {output_path}")
            else:
                print(f"Predicted class index {predicted_class} is out of range.")

# Classify and save images
def classify_and_save_images(images, model, vegetation_classes, filtered_image_folder, n):
    if not os.path.exists(filtered_image_folder):
        os.makedirs(filtered_image_folder)

    for veg_class in vegetation_classes:
        class_folder = os.path.join(filtered_image_folder, veg_class)
        if not os.path.exists(class_folder):
            os.makedirs(class_folder)

    for file_path, img_array in images:
        try:
            img_array_exp = np.expand_dims(img_array, axis=0)
            prediction = model.predict(img_array_exp)
            
            # Debug prints
            print(f"Prediction shape: {prediction.shape}")
            print(f"Prediction values: {prediction}")
            
            predicted_class = np.argmax(prediction, axis=1)[0]
            print(f"Predicted class index: {predicted_class}")
            
            if predicted_class < len(vegetation_classes):
                predicted_label = vegetation_classes[predicted_class]

                if predicted_label in vegetation_classes:
                    # Splice the image if it's a vegetation class
                    cropped_images = splice_image(file_path, n)
                    
                    for cropped_img, idx in cropped_images:
                        base_name = os.path.splitext(os.path.basename(file_path))[0]
                        new_filename = f"{base_name}_part{idx}.jpg"
                        save_path = os.path.join(filtered_image_folder, predicted_label, new_filename)

                        cropped_img.save(save_path)
                        print(f"Spliced image {file_path} classified as {predicted_label} and saved to {save_path}")
            else:
                print(f"Image {file_path} classified as {predicted_label} but not a vegetation class.")

        except Exception as e:
            print(f"Error processing image {file_path}: {e}")

# Load and prepare images for prediction
def load_and_prepare_images(folder):
    image_files = glob.glob(os.path.join(folder, '*.*'))
    images = []
    for file in image_files:
        try:
            with Image.open(file) as img:
                img_array = preprocess_image(img)
                images.append((file, img_array))
        except Exception as e:
            print(f"Error loading image {file}: {e}")
    return images

# Main function
model = load_trained_model(model_path)

# Load images for classification
images = load_and_prepare_images(test_folder)

# Classify and save images
classify_and_save_images(images, model, vegetation_classes, filtered_image_folder, n)

# Analyze image parts
image_files = glob.glob(os.path.join(test_folder, '*.*'))
for image_file in image_files:
    analyze_image_parts(image_file, n, threshold_percentage)

print("Complete")


Model loaded successfully.


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)    │ (None, 64, 64, 3)         │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ zero_padding2d_1              │ (None, 70, 70, 3)         │               0 │ input_layer_1[0][0]        │
│ (ZeroPadding2D)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_53 (Conv2D)            │ (None, 32, 32, 64)        │           9,472 │ zero_padding2d_1[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_53        │ (None, 32, 32, 64)        │             256 │ conv2d_53[0][0]            │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_49 (Activation)    │ (None, 32, 32, 64)        │               0 │ batch_normalization_53[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ average_pooling2d_2           │ (None, 15, 15, 64)        │               0 │ activation_49[0][0]        │
│ (AveragePooling2D)            │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_54 (Conv2D)            │ (None, 15, 15, 64)        │           4,160 │ average_pooling2d_2[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_54        │ (None, 15, 15, 64)        │             256 │ conv2d_54[0][0]            │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_50 (Activation)    │ (None, 15, 15, 64)        │               0 │ batch_normalization_54[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_55 (Conv2D)            │ (None, 15, 15, 64)        │          36,928 │ activation_50[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_55        │ (None, 15, 15, 64)        │             256 │ conv2d_55[0][0]            │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_51 (Activation)    │ (None, 15, 15, 64)        │               0 │ batch_normalization_55[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_56 (Conv2D)            │ (None, 15, 15, 256)       │          16,640 │ activation_51[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_57 (Conv2D)            │ (None, 15, 15, 256)       │          16,640 │ average_pooling2d_2[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_56        │ (None, 15, 15, 256)       │           1,02

 Total params: 23,608,204 (90.06 MB)

 Trainable params: 23,555,082 (89.86 MB)

 Non-trainable params: 53,120 (207.50 KB)

 Optimizer params: 2 (12.00 B)